# Importation des bibliothèques nécessaires
Importer les bibliothèques nécessaires, y compris pandas, numpy, scikit-learn, et matplotlib.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns

# Set seaborn style for plots
sns.set(style="whitegrid")

# Chargement des données
Charger les fichiers CSV HDFS_occurence_matrix_HDFS_train.csv, HDFS_occurence_matrix_HDFS_valid.csv, et HDFS_occurence_matrix_HDFS_test.csv dans des DataFrames pandas.

In [7]:
# Chargement des données
train_df = pd.read_csv('HDFS_results/Event_occurence_matrix_HDFS_train.csv')
valid_df = pd.read_csv('HDFS_results/Event_occurence_matrix_HDFS_valid.csv')
test_df = pd.read_csv('HDFS_results/Event_occurence_matrix_HDFS_test.csv')

# Afficher les premières lignes des DataFrames pour vérifier le chargement
print(train_df.head())
print(valid_df.head())
print(test_df.head())

                    BlockId    Label  Type    Time   Date  E1  E2  E3  E4  E5  \
0   blk_-100000266894974466  Success     0   20106  81110   3   3   3   3   3   
1  blk_-1000014584150379967  Success     0   15117  81110   3   3   3   3   3   
2  blk_-1000028658773048709  Success     0  221431  81110   3   3   3   3   0   
3  blk_-1000054577281647820  Success     0   12444  81110   3   3   3   3   3   
4  blk_-1000083860370843431  Success     0  211524  81109   3   3   3   3   3   

   ...  E20  E21  E22  E23  E24  E25  E26  E27  E28  E29  
0  ...    0    0    0    0    0    0    0    0    0    0  
1  ...    0    0    0    0    0    0    0    0    0    0  
2  ...    0    0    0    0    0    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    0    0    0  
4  ...    0    0    0    0    0    0    0    0    0    0  

[5 rows x 34 columns]
                    BlockId    Label  Type   Time   Date  E1  E2  E3  E4  E5  \
0  blk_-1000028658773048709  Success     0  44331  81111

# Analyse des variables et traitement des corrélations
Calculer la matrice de corrélation sur les variables indépendantes dans l'ensemble d'entraînement. Identifier et supprimer les variables fortement corrélées (corrélation > 0,9).

In [ ]:
# Analyse des variables et traitement des corrélations

# Calcul de la matrice de corrélation sur les variables indépendantes
correlation_matrix = train_df.drop(columns=['BlockId', 'Label', 'Type', 'Time', 'Date'])

# Affichage de la matrice de corrélation
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Matrice de corrélation des variables indépendantes')
plt.show()

# Identification des variables fortement corrélées (corrélation > 0,9)
high_corr_var = set()
threshold = 0.9

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            colname = correlation_matrix.columns[i]
            high_corr_var.add(colname)

# Suppression des variables fortement corrélées
train_df_reduced = train_df.drop(columns=high_corr_var)
valid_df_reduced = valid_df.drop(columns=high_corr_var)
test_df_reduced = test_df.drop(columns=high_corr_var)

# Affichage des variables supprimées
print(f"Variables supprimées en raison d'une forte corrélation: {high_corr_var}")

# Affichage des premières lignes des DataFrames réduits pour vérifier les modifications
print(train_df_reduced.head())
print(valid_df_reduced.head())
print(test_df_reduced.head())

# Création des échantillons avec bootstrap
Utiliser le bootstrap pour créer des échantillons de l'ensemble d'entraînement. Chaque échantillon a la même taille que l'ensemble d'entraînement d'origine (avec remplacement).

In [ ]:
# Création des échantillons avec bootstrap

# Définir le nombre d'échantillons bootstrap
n_bootstrap_samples = 100

# Initialiser une liste pour stocker les échantillons bootstrap
bootstrap_samples = []

# Créer des échantillons bootstrap
for _ in range(n_bootstrap_samples):
    bootstrap_sample = resample(train_df_reduced, replace=True, n_samples=len(train_df_reduced), random_state=42)
    bootstrap_samples.append(bootstrap_sample)

# Afficher le nombre d'échantillons bootstrap créés
print(f"Nombre d'échantillons bootstrap créés: {len(bootstrap_samples)}")

# Afficher les premières lignes du premier échantillon bootstrap pour vérifier
print(bootstrap_samples[0].head())

# Modélisation et validation
Pour chaque échantillon bootstrap, entraîner un modèle de régression logistique sur l'ensemble d'entraînement. Valider ce modèle sur un ensemble de validation distinct.

In [ ]:
# Modélisation et validation

# Initialiser les listes pour stocker les performances des modèles
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
roc_auc_list = []

# Séparer les features et la cible pour l'ensemble de validation
X_valid = valid_df_reduced.drop(columns=['target'])
y_valid = valid_df_reduced['target']

# Pour chaque échantillon bootstrap, entraîner un modèle de régression logistique et valider
for bootstrap_sample in bootstrap_samples:
    # Séparer les features et la cible pour l'échantillon bootstrap
    X_train_bootstrap = bootstrap_sample.drop(columns=['target'])
    y_train_bootstrap = bootstrap_sample['target']
    
    # Entraîner le modèle de régression logistique
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train_bootstrap, y_train_bootstrap)
    
    # Prédire sur l'ensemble de validation
    y_pred = model.predict(X_valid)
    y_pred_proba = model.predict_proba(X_valid)[:, 1]
    
    # Calculer les métriques de performance
    accuracy_list.append(accuracy_score(y_valid, y_pred))
    precision_list.append(precision_score(y_valid, y_pred))
    recall_list.append(recall_score(y_valid, y_pred))
    f1_list.append(f1_score(y_valid, y_pred))
    roc_auc_list.append(roc_auc_score(y_valid, y_pred_proba))

# Afficher les performances moyennes
print(f"Accuracy moyenne: {np.mean(accuracy_list):.4f}")
print(f"Précision moyenne: {np.mean(precision_list):.4f}")
print(f"Rappel moyen: {np.mean(recall_list):.4f}")
print(f"F1-Score moyen: {np.mean(f1_list):.4f}")
print(f"AUC-ROC moyen: {np.mean(roc_auc_list):.4f}")

# Afficher les performances des modèles sous forme de boxplots
plt.figure(figsize=(12, 8))
plt.subplot(2, 3, 1)
sns.boxplot(data=accuracy_list)
plt.title('Accuracy')

plt.subplot(2, 3, 2)
sns.boxplot(data=precision_list)
plt.title('Précision')

plt.subplot(2, 3, 3)
sns.boxplot(data=recall_list)
plt.title('Rappel')

plt.subplot(2, 3, 4)
sns.boxplot(data=f1_list)
plt.title('F1-Score')

plt.subplot(2, 3, 5)
sns.boxplot(data=roc_auc_list)
plt.title('AUC-ROC')

plt.tight_layout()
plt.show()

# Répétition du processus
Répéter les étapes de création d'échantillons bootstrap et de modélisation un nombre défini de fois. Stocker les performances (Précision, Rappel et AUC).

In [ ]:
# Répétition du processus

# Initialiser les listes pour stocker les performances des modèles
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
roc_auc_list = []

# Définir le nombre de répétitions
n_repetitions = 100

# Séparer les features et la cible pour l'ensemble de validation
X_valid = valid_df_reduced.drop(columns=['target'])
y_valid = valid_df_reduced['target']

# Répéter les étapes de création d'échantillons bootstrap et de modélisation
for _ in range(n_repetitions):
    # Créer un échantillon bootstrap
    bootstrap_sample = resample(train_df_reduced, replace=True, n_samples=len(train_df_reduced), random_state=42)
    
    # Séparer les features et la cible pour l'échantillon bootstrap
    X_train_bootstrap = bootstrap_sample.drop(columns=['target'])
    y_train_bootstrap = bootstrap_sample['target']
    
    # Entraîner le modèle de régression logistique
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train_bootstrap, y_train_bootstrap)
    
    # Prédire sur l'ensemble de validation
    y_pred = model.predict(X_valid)
    y_pred_proba = model.predict_proba(X_valid)[:, 1]
    
    # Calculer les métriques de performance
    accuracy_list.append(accuracy_score(y_valid, y_pred))
    precision_list.append(precision_score(y_valid, y_pred))
    recall_list.append(recall_score(y_valid, y_pred))
    f1_list.append(f1_score(y_valid, y_pred))
    roc_auc_list.append(roc_auc_score(y_valid, y_pred_proba))

# Afficher les performances moyennes
print(f"Accuracy moyenne: {np.mean(accuracy_list):.4f}")
print(f"Précision moyenne: {np.mean(precision_list):.4f}")
print(f"Rappel moyen: {np.mean(recall_list):.4f}")
print(f"F1-Score moyen: {np.mean(f1_list):.4f}")
print(f"AUC-ROC moyen: {np.mean(roc_auc_list):.4f}")

# Afficher les performances des modèles sous forme de boxplots
plt.figure(figsize=(12, 8))
plt.subplot(2, 3, 1)
sns.boxplot(data=accuracy_list)
plt.title('Accuracy')

plt.subplot(2, 3, 2)
sns.boxplot(data=precision_list)
plt.title('Précision')

plt.subplot(2, 3, 3)
sns.boxplot(data=recall_list)
plt.title('Rappel')

plt.subplot(2, 3, 4)
sns.boxplot(data=f1_list)
plt.title('F1-Score')

plt.subplot(2, 3, 5)
sns.boxplot(data=roc_auc_list)
plt.title('AUC-ROC')

plt.tight_layout()
plt.show()

# Évaluation finale sur le test
Appliquer le meilleur modèle sur l'ensemble de test. Calculer la matrice de confusion et des métriques comme l'accuracy, la précision, le rappel, le F1-Score, et l'AUC-ROC.

In [ ]:
# Évaluation finale sur le test

# Séparer les features et la cible pour l'ensemble de test
X_test = test_df_reduced.drop(columns=['target'])
y_test = test_df_reduced['target']

# Entraîner le meilleur modèle (Régression logistique) sur l'ensemble d'entraînement complet
best_model = LogisticRegression(max_iter=1000, random_state=42)
best_model.fit(train_df_reduced.drop(columns=['target']), train_df_reduced['target'])

# Prédire sur l'ensemble de test
y_test_pred = best_model.predict(X_test)
y_test_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Calculer les métriques de performance
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred_proba)

# Afficher les métriques de performance
print(f"Accuracy sur le test: {accuracy_test:.4f}")
print(f"Précision sur le test: {precision_test:.4f}")
print(f"Rappel sur le test: {recall_test:.4f}")
print(f"F1-Score sur le test: {f1_test:.4f}")
print(f"AUC-ROC sur le test: {roc_auc_test:.4f}")

# Calculer la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_test_pred)

# Afficher la matrice de confusion
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'])
plt.xlabel('Prédiction')
plt.ylabel('Réalité')
plt.title('Matrice de confusion')
plt.show()

# Tracer la courbe ROC
fpr, tpr, _ = roc_curve(y_test, y_test_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'AUC-ROC = {roc_auc_test:.4f}')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlabel('Taux de faux positifs (FPR)')
plt.ylabel('Taux de vrais positifs (TPR)')
plt.title('Courbe ROC')
plt.legend(loc="lower right")
plt.show()

# Application à d'autres algorithmes (Random Forest)
Reprendre les étapes de modélisation et de validation pour un Random Forest. Comparer les performances des deux modèles sur les mêmes métriques.

In [ ]:
# Application à d'autres algorithmes (Random Forest)

# Initialiser les listes pour stocker les performances des modèles Random Forest
rf_accuracy_list = []
rf_precision_list = []
rf_recall_list = []
rf_f1_list = []
rf_roc_auc_list = []

# Pour chaque échantillon bootstrap, entraîner un modèle Random Forest et valider
for bootstrap_sample in bootstrap_samples:
    # Séparer les features et la cible pour l'échantillon bootstrap
    X_train_bootstrap = bootstrap_sample.drop(columns=['target'])
    y_train_bootstrap = bootstrap_sample['target']
    
    # Entraîner le modèle Random Forest
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train_bootstrap, y_train_bootstrap)
    
    # Prédire sur l'ensemble de validation
    y_pred = rf_model.predict(X_valid)
    y_pred_proba = rf_model.predict_proba(X_valid)[:, 1]
    
    # Calculer les métriques de performance
    rf_accuracy_list.append(accuracy_score(y_valid, y_pred))
    rf_precision_list.append(precision_score(y_valid, y_pred))
    rf_recall_list.append(recall_score(y_valid, y_pred))
    rf_f1_list.append(f1_score(y_valid, y_pred))
    rf_roc_auc_list.append(roc_auc_score(y_valid, y_pred_proba))

# Afficher les performances moyennes pour Random Forest
print(f"Random Forest - Accuracy moyenne: {np.mean(rf_accuracy_list):.4f}")
print(f"Random Forest - Précision moyenne: {np.mean(rf_precision_list):.4f}")
print(f"Random Forest - Rappel moyen: {np.mean(rf_recall_list):.4f}")
print(f"Random Forest - F1-Score moyen: {np.mean(rf_f1_list):.4f}")
print(f"Random Forest - AUC-ROC moyen: {np.mean(rf_roc_auc_list):.4f}")

# Afficher les performances des modèles Random Forest sous forme de boxplots
plt.figure(figsize=(12, 8))
plt.subplot(2, 3, 1)
sns.boxplot(data=rf_accuracy_list)
plt.title('Random Forest - Accuracy')

plt.subplot(2, 3, 2)
sns.boxplot(data=rf_precision_list)
plt.title('Random Forest - Précision')

plt.subplot(2, 3, 3)
sns.boxplot(data=rf_recall_list)
plt.title('Random Forest - Rappel')

plt.subplot(2, 3, 4)
sns.boxplot(data=rf_f1_list)
plt.title('Random Forest - F1-Score')

plt.subplot(2, 3, 5)
sns.boxplot(data=rf_roc_auc_list)
plt.title('Random Forest - AUC-ROC')

plt.tight_layout()
plt.show()

# Évaluation finale sur le test pour Random Forest

# Entraîner le meilleur modèle Random Forest sur l'ensemble d'entraînement complet
best_rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
best_rf_model.fit(train_df_reduced.drop(columns=['target']), train_df_reduced['target'])

# Prédire sur l'ensemble de test
y_test_pred_rf = best_rf_model.predict(X_test)
y_test_pred_proba_rf = best_rf_model.predict_proba(X_test)[:, 1]

# Calculer les métriques de performance pour Random Forest
accuracy_test_rf = accuracy_score(y_test, y_test_pred_rf)
precision_test_rf = precision_score(y_test, y_test_pred_rf)
recall_test_rf = recall_score(y_test, y_test_pred_rf)
f1_test_rf = f1_score(y_test, y_test_pred_rf)
roc_auc_test_rf = roc_auc_score(y_test, y_test_pred_proba_rf)

# Afficher les métriques de performance pour Random Forest
print(f"Random Forest - Accuracy sur le test: {accuracy_test_rf:.4f}")
print(f"Random Forest - Précision sur le test: {precision_test_rf:.4f}")
print(f"Random Forest - Rappel sur le test: {recall_test_rf:.4f}")
print(f"Random Forest - F1-Score sur le test: {f1_test_rf:.4f}")
print(f"Random Forest - AUC-ROC sur le test: {roc_auc_test_rf:.4f}")

# Calculer la matrice de confusion pour Random Forest
conf_matrix_rf = confusion_matrix(y_test, y_test_pred_rf)

# Afficher la matrice de confusion pour Random Forest
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_rf, annot=True, fmt="d", cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'])
plt.xlabel('Prédiction')
plt.ylabel('Réalité')
plt.title('Random Forest - Matrice de confusion')
plt.show()

# Tracer la courbe ROC pour Random Forest
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_test_pred_proba_rf)
plt.figure(figsize=(8, 6))
plt.plot(fpr_rf, tpr_rf, color='blue', lw=2, label=f'Random Forest AUC-ROC = {roc_auc_test_rf:.4f}')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlabel('Taux de faux positifs (FPR)')
plt.ylabel('Taux de vrais positifs (TPR)')
plt.title('Random Forest - Courbe ROC')
plt.legend(loc="lower right")
plt.show()